![Logo TUBAF](https://tu-freiberg.de/sites/default/files/media/freiberger-alumni-netzwerk-6127/wbm_orig_rgb_0.jpg)

Exercise material of the MSc-level course **Foundations of Soil Mechanics**.
Held at Technische Universität Bergakademie Freiberg.

Comments to:

*Prof. Dr. Thomas Nagel  
Chair of Soil Mechanics and Foundation Engineering  
Geotechnical Institute  
Technische Universität Bergakademie Freiberg.*

https://tu-freiberg.de/en/soilmechanics


# Active Earth Pressure Coulomb

![Erddruck.png](https://raw.githubusercontent.com/nagelt/Teaching_Scripts/master/Images/Erddruck.png)

## Geometry 
Let the retaining wall have a vertical height $H$ and the origin be placed at the bottom of the slip surface which has length $L$. Then, the problem is characterized by the following points in the $x-z$ plane:

* Origin: $O = [0,0]$
* Top left: $A = [-H \tan \alpha, H]$
* Top right: $B = [L\cos \vartheta, L \sin \vartheta]$

The length of the slip surface can be found via the following trigonometric relation:

\begin{align}
    L \sin \vartheta &= (L \cos \vartheta + H \tan\alpha) \tan \beta
\end{align}

Thus we arrive at

\begin{align}
    L &= \frac{H(1 + \tan \alpha \tan \beta)}{\sin \vartheta - \cos \vartheta \tan \beta}
\end{align}

The area of the triangle (i.e. the volume of the slip body per meter) can also be calculated. We use Gauß' equation for that:

$$
    A = \frac{1}{2} \left| (x_2 y_3 - x_3 y_2) - (x_1 y_3 - x_3 y_1) + (x_1 y_2 - x_2 y_1) \right|
$$

Let Python do the math for us ...

In [1]:
#HIDDEN
import numpy as np #numerical methods
#import sympy as sp #symbolic operations
import matplotlib.pyplot as plt #plotting
import sympy as sp
sp.init_printing(use_latex='mathjax') #makes sympy output look nice

#Some plot settings
import plot_functions.plot_settings
%run plot_functions/earth_pressure_plots.ipynb

We first define our geometrical properties:

In [2]:
H,alpha,beta,theta = sp.symbols('H alpha beta vartheta')

In [3]:
L = H*(1+sp.tan(alpha)*sp.tan(beta))/(sp.sin(theta) - sp.cos(theta) * sp.tan(beta))
L

       H⋅(tan(α)⋅tan(β) + 1)        
────────────────────────────────────
sin(vartheta) - cos(vartheta)⋅tan(β)

We now define the function that calculates the area of the triangle:

In [4]:
def triangle_area_Gauss(A,B,C):
    term1 = (B[0]*C[1] - C[0] * B[1])
    term2 = (A[0]*C[1] - C[0] * A[1])
    term3 = (A[0]*B[1] - B[0] * A[1])
    return -(term1 - term2 + term3)/2

The points we defined above can now be inserted into this equation:

In [5]:
PO = [0,0]
PA = [- H * sp.tan(alpha), H]
PB = [L * sp.cos(theta), L*sp.sin(theta)]
A = triangle_area_Gauss(PO,PA,PB).simplify()
A

 2                                                           
H ⋅(sin(vartheta)⋅tan(α) + cos(vartheta))⋅(tan(α)⋅tan(β) + 1)
─────────────────────────────────────────────────────────────
           2⋅(sin(vartheta) - cos(vartheta)⋅tan(β))          

Thus, the area of our triangle is given as:

$$
    A = \frac{H^2 (\sin \vartheta \tan \alpha + \cos \vartheta) (\tan \alpha \tan \beta + 1)}{2(\sin \vartheta - \cos \vartheta \tan \beta)}
$$

Now we can write the force balance. For that purpose, we introduce the material's specific weight $\gamma$, friction angle $\varphi$ and the wall friction angle $\delta_\text{a}$ as well as the cohesion $c$.

In [6]:
c, delta, phi, gamma = sp.symbols('c delta_a varphi gamma')

We can now calculate the downward force due to gravity as well as the cohesive force along the slip surface. The active earth pressure force $E_\text{a}$ and the reaction force $Q$ in the slip line are introduced as unknowns:

In [7]:
Ea, Q, G, C = sp.symbols('E_a Q G C')

We now write the vertical and horizontal equilibrium conditions:

\begin{align}
    \text{horizontal:}\quad  0 &= C \cos\vartheta - Q \sin (\vartheta - \varphi) + E_\text{a} \cos (\alpha + \delta_\text{a})
    \\
    \text{vertikal:}\quad  0 &= C \sin\vartheta + Q \cos (\vartheta - \varphi) + E_\text{a} \sin (\alpha + \delta_\text{a}) - G
\end{align}

In [8]:
Fh = sp.Eq(C * sp.cos(theta) - Q * sp.sin(theta - phi) + Ea * sp.cos(alpha + delta),0)
Fv = sp.Eq(C * sp.sin(theta) + Q * sp.cos(theta - phi) + Ea * sp.sin(alpha + delta) - G,0)

In [9]:
Fh

C⋅cos(vartheta) + Eₐ⋅cos(α + δₐ) + Q⋅sin(varphi - vartheta) = 0

In [10]:
Fv

C⋅sin(vartheta) + Eₐ⋅sin(α + δₐ) - G + Q⋅cos(varphi - vartheta) = 0

In [11]:
result = sp.solve([Fh,Fv],(Ea,Q))

In [12]:
result

⎧                     C⋅sin(vartheta)⋅sin(varphi - vartheta)                                     C⋅cos(vartheta)⋅cos(v
⎨Eₐ: ──────────────────────────────────────────────────────────────────────── - ──────────────────────────────────────
⎩    -sin(α + δₐ)⋅sin(varphi - vartheta) + cos(α + δₐ)⋅cos(varphi - vartheta)   -sin(α + δₐ)⋅sin(varphi - vartheta) + 

arphi - vartheta)                                            G⋅sin(varphi - vartheta)                                 
────────────────────────────────── - ────────────────────────────────────────────────────────────────────────, Q: - ──
cos(α + δₐ)⋅cos(varphi - vartheta)   -sin(α + δₐ)⋅sin(varphi - vartheta) + cos(α + δₐ)⋅cos(varphi - vartheta)       -s

                    C⋅sin(vartheta)⋅cos(α + δₐ)                                                C⋅sin(α + δₐ)⋅cos(varth
────────────────────────────────────────────────────────────────────── + ─────────────────────────────────────────────
in(α + δₐ)⋅sin(varphi - vartheta) + cos(α + δₐ

Gravitational and cohesive forces can be determined based on material properties and geometry:

In [13]:
result.get(Ea).subs(C,0).subs(G,A*gamma).simplify()

  2                                                                                     
-H ⋅γ⋅(sin(vartheta)⋅tan(α) + cos(vartheta))⋅(tan(α)⋅tan(β) + 1)⋅sin(varphi - vartheta) 
────────────────────────────────────────────────────────────────────────────────────────
        2⋅(sin(vartheta) - cos(vartheta)⋅tan(β))⋅cos(α + δₐ + varphi - vartheta)        

Now we would have to find the maximum with respect to $\vartheta$ in order to identify the active earth pressure and the critical slip angle $\vartheta_\text{a}$. The equations get very complex and we decide to go for a numerical solution.

In [14]:
plot_interactive_earth_pressure()

interactive(children=(IntSlider(value=10, description='$c$ / kN m$^{-2}$:', max=50, step=5), IntSlider(value=3…

## Tasks

* Compare the numbers you get to results obtained from the analytical expressions you find in the current norm.
* Investigate the influence of the different parameters.
* What happens at very high cohesion values? What does this mean?
* How high can a cohesive vertical slope be roughly? Select suitable parameters to answer this question.


![Erddruck.png](https://raw.githubusercontent.com/nagelt/Teaching_Scripts/master/Images/Erddruck.png)